# GPTQ: Accurate Post-Training Quantization for Generative Pre-trained Transformers

En el paper [GPTQ: Accurate Post-Training Quantization for Generative Pre-trained Transformers](https://arxiv.org/abs/2210.17323) se expone la necesidad de crear un método de cuantización post entrenamiento que no degrade la calidad del modelo. En este post hemos visto el método [llm.int8()](https://maximofn.com/llm-int8/) que cuantiza a INT8 algunos vectores de las matrices de pesos, siempre y cuando ninguno de sus valores sobrepase un valor umbral, lo cual está muy bien, pero no cuantizan todos los pesos del modelo. En este paper proponen un método que cuantiza todos los pesos del modelo a 4 y 3 bits, sin degradar la calidad del modelo. Lo que supone un ahorro considerable de memoria, no solo porque se cuantizan todos los pesos, sino porque además se hace a 4 y 3 bits, en vez de a 8 bits.

## Trabajos en los que se basa

### Cuantización por capas

Por un lado se basan en los trabajos `Nagel et al., 2020`; `Wang et al., 2020`; `Hubara et al., 2021` y `Frantar et al., 2022`, que proponen cuantizar los pesos de las capas de una red neuronal a 4 y 3 bits, sin degradar la calidad del modelo.

Teniendo un conjunto de datos `m` proveniente de un dataset, a cada capa `l` se le meten los datos y se obtiene la salida de los pesos `W` de dicha capa. Así que lo que se hace es buscar unos pesos nuevos `Ŵ` cuantizados que minimicen el error cuadrático en relación con la salida de la capa de precisión total

`argmin_Ŵ||WX− ŴX||^2`

Los valores de `Ŵ` se establecen antes de realizar el proceso de cuantización y durante el proceso, cada parámetro de `Ŵ` puede cambiar de valor independientemente sin depender del valor de los demás parámetros de `Ŵ`.

### Optimal brain quantization (OBQ)

En el trabajo de `OBQ` de `Frantar et al., 2022` optimizan el proceso de cuantización por capas anterior, haciendo que llegue a ser hasta 3 veces más rápido. Esto ayuda con los modelos grandes, ya que cuantizar un modelo grande puede llevar mucho tiempo.

El método `OBQ` es un enfoque para resolver el problema de cuantificación por capas en modelos de lenguaje. `OBQ` parte de la idea de que el error cuadrático se puede descomponer en la suma de errores individuales para cada fila de la matriz de pesos. Luego, el método cuantifica cada peso de manera independiente, actualizando siempre los pesos no cuantificados para compensar el error incurrido por la cuantificación.

El método es capaz de cuantificar modelos de tamaño mediano en tiempos razonables, pero como es un algoritmo de complejidad cúbica hace que sea extremadamente costoso aplicarlo a modelos con miles de millones de parámetros.

## Algoritmo de GPTQ

### Paso 1: Información de orden arbitrario

En `OBQ` se buscaba la fila de pesos que creaba menor error cuadrático medio para cuantizar, pero se dieron cuenta que al hacerlo de manera aleatoria no aumentaba mucho el error cuadrático medio final. Por lo que en vez de buscar la fila que minimiza el error cuadrático medio, que creaba una complejidad cúbica en el algoritmo, se hace siempre en el mismo orden. Gracias a esto se reduce mucho el tiempo de ejecución del algoritmo de cuantización

### Paso 2: Actualizaciones lazy batch

Como la actualización de los pesos se hace fila a fila, hace que sea un proceso lento y no se aproveche del todo el hardware. Por lo que proponen realizar las actualizaciones en lotes de `B=128` filas. De esta manera se aprovecha mejor el hardware y se reduce el tiempo de ejecución del algoritmo.

### Paso 3: Reformulación de Cholesky

El problema de hacer las actualizaciones por lotes es que, debido a la gran escala de los modelos, se pueden producir errores numéricos que afectan la precisión del algoritmo. Concretamente, se pueden obtener matrices indefinidas, lo que provoca que el algoritmo actualice los pesos restantes en direcciones incorrectas, lo que resulta en una cuantización muy mala.

Para solucionar esto, los autores del paper proponen utilizar una reformulación de Cholesky, que es un método más numéricamente estable.